# Задача 1

## Пункт a

У нас игра, где есть два игрока:

Игрок $1$: penalty_taker, или пенальтист, или такер

Игрок $2$: goalkeeper, или вратарь, или Аркадий Иванов, тк он у нас один $:)$

У нас есть такие стратегии для кажд. игрока: $S_1 = S_2 = \{L, R\}$, где, понятно, что $L$ - влево, а $R$ - вправо

Из условия знаем, что выигрыш пенальтиста - это вероятность забить гол, а выигрыш вратаря - это 1 - вер. гола

Напишем теперь код на python, который и строит эти матрицы выигрышей. Для каждого игрока мы находим количество всего попыток, которые он совершил и количество успешных попыток, то есть голов, которые он забил, ну и это мы делаем для всех возможных направлений, а вероятность мы считаем как собственно деление кол-во всего попыток на кол-во успешных попыток, что очев

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("data_penalty_results.xlsx", sheet_name=None)


def calc_matrix(data):
    tot_tries = np.zeros((2, 2))
    suc_tries = np.zeros((2, 2))
    dirs = {'L': 0, 'R': 1}

    for _, row in data.iterrows():
        taker_dir = row['penalty taker action']
        keeper_dir = row['goalkeeper action']
        outcome = row['result']

        taker_idx = dirs[taker_dir]
        keeper_idx = dirs[keeper_dir]

        tot_tries[taker_idx, keeper_idx] += 1
        suc_tries[taker_idx, keeper_idx] += outcome

    payoff_matrix = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            payoff_matrix[i, j] = suc_tries[i, j] / tot_tries[i, j]

    return payoff_matrix


for player_name, player_data in df.items():
    print(f"\nИгрок: {player_name}")
    print("-" * 35)

    payoff = calc_matrix(player_data)

    print("Матрица выигрышей:")
    print("          Вратарь L   Вратарь R")
    print(f"Такер L   {payoff[0, 0]:.3f}       {payoff[0, 1]:.3f}")
    print(f"Такер R   {payoff[1, 0]:.3f}       {payoff[1, 1]:.3f}")

    taker_L = "L" if payoff[0, 0] > payoff[1, 0] else "R"
    taker_R = "L" if payoff[0, 1] > payoff[1, 1] else "R"

    keeper_L = "L" if payoff[0, 0] < payoff[0, 1] else "R"
    keeper_R = "L" if payoff[1, 0] < payoff[1, 1] else "R"

    nash = []

    if taker_L == "L" and keeper_L == "L":
        nash.append("(Такер L, Вратарь L)")
    if taker_R == "R" and keeper_R == "R":
        nash.append("(Такер R, Вратарь R)")
    if taker_L == "R" and keeper_R == "L":
        nash.append("(Такер R, Вратарь L)")
    if taker_R == "L" and keeper_L == "R":
        nash.append("(Такер L, Вратарь R)")

    if nash:
        print(f"Есть равновесии по Нэшу, вот они: {', '.join(nash)}")
    else:
        print("Нет равновесий по Нэшу")

    print()


Игрок: Dennis Kolos
-----------------------------------
Матрица выигрышей:
          Вратарь L   Вратарь R
Такер L   0.190       0.630
Такер R   0.910       0.160
Нет равновесий по Нэшу


Игрок: Ilya Tikhonov
-----------------------------------
Матрица выигрышей:
          Вратарь L   Вратарь R
Такер L   0.070       0.790
Такер R   0.830       0.160
Нет равновесий по Нэшу


Игрок: Boris Saveliev
-----------------------------------
Матрица выигрышей:
          Вратарь L   Вратарь R
Такер L   0.250       0.980
Такер R   0.800       0.075
Нет равновесий по Нэшу


Игрок: Yurii Ivanov
-----------------------------------
Матрица выигрышей:
          Вратарь L   Вратарь R
Такер L   0.280       0.750
Такер R   0.600       0.085
Нет равновесий по Нэшу


Игрок: Yan Nay
-----------------------------------
Матрица выигрышей:
          Вратарь L   Вратарь R
Такер L   0.420       0.740
Такер R   0.620       0.150
Нет равновесий по Нэшу



Вот тут мы можем видеть результаты собственно, я решил сразу же в коде реализовать поиск равновесий по Нэшу в чист. стратегиях, то есть я нашел лучшие ответы пенальтиста на опред. ход вратаря и также лучшие реакции вратаря на опред. ход пенальтиста, далее проверил все возможные комбинации стратегий на поиск равновесий по Нэшу и как выяснилось в каждом варианте кому-то да выгодно уклониться

Ну и как мы увидели, чистых равновесий нет, что в целом очев для такой игры $:)$

## Пункт б

В целом, я уже написал код, который ищет чистые равновесия, но распишу руками, хорошо $:)$

### Илья Тихонов:

$$
\begin{pmatrix}
  0.07 & 0.79 \\
  0.83 & 0.16
\end{pmatrix}
$$

Проверим просто все 4 комбинации _**(чистые стратегии)**_, то есть перебором:

- $(L, L)$, такеру выгодно отклониться, так как тут он получает $0.07$, а при $R$ получил бы $0.79$, что больше
- $(L, R)$, если вратарь отклоняется к $L$, то такер получает меньше: $0.07 < 0.79$ $⟹$ выгодно отклониться
- $(R, L)$, если вратарь отклоняется к $R$, то такер опять получает меньше: $0.16 < 0.83$ $⟹$ опять выгодно отклониться вратарю
- $(R, R)$, если такер отклонится к $L$, то он получит больше: $0.79 > 0.16$, то есть ему выгодно отклонение

Так мы поняли, что равновесий нет $:($

Теперь **смешанные стратегии**:

Пусть $p$ - вер., с которой такер выбирает $L$, а $q$ - с которой вратарь выбирает $L$

Запишем ожид. выигрыши для пенальтиста:

- если он выбирает $L$:

$$
\mathbb{E}_L^{taker} = 0.07q + 0.79(1 - q) = 0.79 - 0.72q
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{taker} = 0.83q + 0.16(1 - q) = 0.16 + 0.67q
$$

Ну и очев, что он безразличен между $L, R$, когда $\mathbb{E}_L = \mathbb{E}_R$:

$$
0.79 - 0.72q = 0.16 + 0.67q ⟹ q^* = \frac{63}{139}
$$

Теперь запишем ожид. выигрыши для вратаря:

- если он выбирает $L$:

$$
\mathbb{E}_L^{keeper} = 0.93p + 0.17(1 - p) = 0.17 + 0.76p
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{keeper} = 0.21p + 0.84(1 - p) = 0.84 - 0.63p
$$

Ну и он безразличен, когда $\mathbb{E}_L = \mathbb{E}_R$:

$$
0.17 + 0.76p = 0.84 - 0.63p ⟹ p^* = \frac{67}{139}
$$

Ну вот мы и нашли равновесие в смешанных для Ильи Тихонова $:)$, это $q^* = \frac{63}{139}$ и $p^* = \frac{67}{139}$

### Борис Савельев

Аналогично все рассчитаем и решим для него $:)$

$$
\begin{pmatrix}
  0.25 & 0.98 \\
  0.8 & 0.075
\end{pmatrix}
$$

Проверим просто все 4 комбинации _**(чистые стратегии)**_, то есть перебором:

- $(L, L)$, такеру выгодно отклониться, так как тут он получает $0.25$, а при $R$ получил бы $0.8$, что больше
- $(L, R)$, если вратарь отклоняется к $L$, то такер получает меньше: $0.25 < 0.98$ $⟹$ выгодно отклониться
- $(R, L)$, если вратарь отклоняется к $R$, то такер опять получает меньше: $0.075 < 0.8$ $⟹$ опять выгодно отклониться вратарю
- $(R, R)$, если такер отклонится к $L$, то он получит больше: $0.98 > 0.075$, то есть ему выгодно отклонение

Так мы поняли, что равновесий нет $:($

Теперь **смешанные стратегии**:

Пусть $p$ - вер., с которой такер выбирает $L$, а $q$ - с которой вратарь выбирает $L$

Запишем ожид. выигрыши для пенальтиста:

- если он выбирает $L$:

$$
\mathbb{E}_L^{taker} = 0.25q + 0.98(1 - q)
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{taker} = 0.8q+0.075(1- q)
$$

Ну и очев, что он безразличен между $L, R$, когда $\mathbb{E}_L = \mathbb{E}_R$:

$$
-0.73q + 0.98= 0.725q + 0.075 ⟹ q^* = \frac{181}{291}
$$

Теперь запишем ожид. выигрыши для вратаря:

- если он выбирает $L$:

$$
\mathbb{E}_L^{keeper} = 0.25p + 0.8(1 - p)
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{keeper} = 0.98p + 0.075(1 - p)
$$

Ну и он безразличен, когда $\mathbb{E}_L = \mathbb{E}_R$:

$$
-0.55p + 0.8 = 0.905p + 0.075 ⟹ p^* = \frac{145}{291}
$$

Ну и мы получили, что в смеш. стратегиях у Савельева равновесие при $p^* = \frac{145}{291}$ и $q^* = \frac{181}{291}$

## Пункт c

### Илья Тихонов:

Знаем, что $q = \frac{1}{2}$, тогда рассчитаем то рассчитаем ожид. выигрыши для пенальтиста по тем формулам, которые я написал выше в пункте б:

- если он выбирает $L$:

$$
\mathbb{E}_L^{taker} = 0.07q + 0.79(1 - q) = 0.79 - 0.72 \cdot \frac 1 2 = 0.43
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{taker} = 0.83q + 0.16(1 - q) = 0.16 + 0.67 \cdot \frac 1 2 = 0.495
$$

Ну тут становится понятно, что пенальтисту Тихонову лучше выбирать стратегию $R$, там мат. ожид. выше $:)$

### Борис Савельев:

Знаем, что $q = \frac{1}{2}$, тогда рассчитаем то рассчитаем ожид. выигрыши для пенальтиста по тем формулам, которые я написал выше в пункте б:

- если он выбирает $L$:

$$
\mathbb{E}_L^{taker} = 0.25 \cdot \frac 1 2 + 0.98 \left(1 - \frac 1 2 \right) = 0.615
$$

- если он выбирает $R$:

$$
\mathbb{E}_R^{taker} = 0.8 \cdot \frac 1 2 + 0.075 \left(1 - \frac 1 2 \right) = 0.4375
$$

Ну тут становится понятно, что пенальтисту Савельева лучше выбирать стратегию $L$, там мат. ожид. выше $:)$

Ответ: все ответы я указал выше

# Задача 2

In [2]:
import pandas as pd

p1 = pd.read_excel("data_payoff_matrix.xlsx", sheet_name="Player 1", index_col=0)
p2 = pd.read_excel("data_payoff_matrix.xlsx", sheet_name="Player 2", index_col=0)

m1 = p1.values
m2 = p2.values

res = []

for i in range(m1.shape[0]):
    for j in range(m1.shape[1]):
        p1_optimal = all(m1[i, j] >= m1[x, j] for x in range(m1.shape[0]))
        p2_optimal = all(m2[i, j] >= m2[i, y] for y in range(m2.shape[1]))

        if p1_optimal and p2_optimal:
            res.append((f"A{i + 1}", f"B{j + 1}", m1[i, j], m2[i, j]))

if res:
    print("Имеем вот такие равновесия в чист. стратегиях:\n")
    for r in res:
        print(f"Стратегия игрока 1: {r[0]}, Стратегия игрока 2: {r[1]}")
        print(f"При этих стратегиях выигрыш игрока 1: {r[2]}, а выигрыш игрока 2: {r[3]}\n")

Имеем вот такие равновесия в чист. стратегиях:

Стратегия игрока 1: A7, Стратегия игрока 2: B17
При этих стратегиях выигрыш игрока 1: 78, а выигрыш игрока 2: 94

Стратегия игрока 1: A10, Стратегия игрока 2: B13
При этих стратегиях выигрыш игрока 1: 98, а выигрыш игрока 2: 98



Тут мы решаем втупую перебором, для каждой пары стратегий $(a_i, b_j)$ мы проверяем является ли $a_i$ лучшим ответом на стратегию $b_j$ игрока 2, и еще проверяем является ли $b_j$ лучшей реакцией игрока 2 на стратегию $a_i$ игрока 1 и если эти оба условия выполняются, то это равновесие и мы его добавляем в список и после выводим, все просто в общем $:)$